<table class="ee-notebook-buttons" align="left">
    <td><a target="_blank"  href="https://github.com/giswqs/earthengine-py-notebooks/tree/master/Image/image_vis.ipynb"><img width=32px src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" /> View source on GitHub</a></td>
    <td><a target="_blank"  href="https://nbviewer.jupyter.org/github/giswqs/earthengine-py-notebooks/blob/master/Image/image_vis.ipynb"><img width=26px src="https://upload.wikimedia.org/wikipedia/commons/thumb/3/38/Jupyter_logo.svg/883px-Jupyter_logo.svg.png" />Notebook Viewer</a></td>
    <td><a target="_blank"  href="https://mybinder.org/v2/gh/giswqs/earthengine-py-notebooks/master?filepath=Image/image_vis.ipynb"><img width=58px src="https://mybinder.org/static/images/logo_social.png" />Run in binder</a></td>
    <td><a target="_blank"  href="https://colab.research.google.com/github/giswqs/earthengine-py-notebooks/blob/master/Image/image_vis.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" /> Run in Google Colab</a></td>
</table>

## Install Earth Engine API
Install the [Earth Engine Python API](https://developers.google.com/earth-engine/python_install) and [geehydro](https://github.com/giswqs/geehydro). The **geehydro** Python package builds on the [folium](https://github.com/python-visualization/folium) package and implements several methods for displaying Earth Engine data layers, such as `Map.addLayer()`, `Map.setCenter()`, `Map.centerObject()`, and `Map.setOptions()`.
The magic command `%%capture` can be used to hide output from a specific cell.

In [ ]:
# %%capture
# !pip install earthengine-api
# !pip install geehydro

Import libraries

In [ ]:
import ee
import folium
import geehydro

Authenticate and initialize Earth Engine API. You only need to authenticate the Earth Engine API once. Uncomment the line `ee.Authenticate()` 
if you are running this notebook for this first time or if you are getting an authentication error.  

In [ ]:
# ee.Authenticate()
ee.Initialize()

## Create an interactive map 
This step creates an interactive map using [folium](https://github.com/python-visualization/folium). The default basemap is the OpenStreetMap. Additional basemaps can be added using the `Map.setOptions()` function. 
The optional basemaps can be `ROADMAP`, `SATELLITE`, `HYBRID`, `TERRAIN`, or `ESRI`.

In [ ]:
Map = folium.Map(location=[40, -100], zoom_start=4)
Map.setOptions('HYBRID')

## Add Earth Engine Python script 

In [ ]:
image = ee.Image('LANDSAT/LC8_L1T_TOA/LC80440342014077LGN00')
# print(image.getInfo())
Map.setCenter(-122.1899, 37.5010, 10)
vis = {'bands': ['B5', 'B4', 'B3'], 'min': 0, 'max': 0.5, 'gamma': [0.95, 1.1, 1]}
# Map.addLayer(image, vis)


# Color palettes
image = ee.Image('LANDSAT/LC8_L1T_TOA/LC80440342014077LGN00')
ndwi = image.normalizedDifference(['B3', 'B5'])
ndwiViz = {'min': 0.5, 'max': 1, 'palette': ['00FFFF', '0000FF']}
# Map.addLayer(ndwi, ndwiViz, 'NDWI')

# Masking
ndwiMasked = ndwi.updateMask(ndwi.gte(0.4))
# Map.addLayer(ndwiMasked, ndwiViz, 'NDWI masked')

# Create visualization layers.
imageRGB = image.visualize({'bands': ['B5', 'B4', 'B3'], max: 0.5})
ndwiRGB = ndwiMasked.visualize({
    'min': 0.5,
    'max': 1,
    'palette': ['00FFFF', '0000FF']
})

# Mosaic the visualization layers and display( or export).
# mosaic = ee.ImageCollection([imageRGB, ndwiRGB]).mosaic()
# mosaic = ee.ImageCollection([image,ndwiMasked]).mosaic()
# Map.addLayer(mosaic, {}, 'mosaic')


roi = ee.Geometry.Point([-122.4481, 37.7599]).buffer(20000)
Map.addLayer(image.clip(roi), vis, 'Landsat 8')
Map.addLayer(ndwiMasked.clip(roi),ndwiViz, 'NDWI')



## Display Earth Engine data layers 

In [ ]:
Map.setControlVisibility(layerControl=True, fullscreenControl=True, latLngPopup=True)
Map